In [117]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostRegressor
from sklearn.model_selection import StratifiedKFold
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error
warnings.filterwarnings('ignore')

In [118]:
train = pd.read_csv('/kaggle/input/new-hd-ai-challenge/train.csv')
test = pd.read_csv('/kaggle/input/new-hd-ai-challenge/test.csv')
sub = pd.read_csv('/kaggle/input/new-hd-ai-challenge/sample_submission.csv')

In [119]:
train["BREADTH"].unique()

array([30., 50., 20., 40., 10., 60., nan])

In [120]:
train["DEPTH"].unique()

array([10., 30., 20.,  0., nan])

In [121]:
train["DRAUGHT"].unique()

array([10., 20.,  0., nan])

In [122]:
train.iloc[356484, train.columns.get_loc("BREADTH")] = 10
train.iloc[356484, train.columns.get_loc("DEPTH")] = 10
train.iloc[356484, train.columns.get_loc("DRAUGHT")] = 0
train.iloc[356484, train.columns.get_loc("LENGTH")] = 70

### 선박 Type별로 따로 모델링

trainCa = train[train['SHIP_TYPE_CATEGORY'] == 'Cargo'] 

testCa = test[test['SHIP_TYPE_CATEGORY'] == 'Cargo'] 

trainCa

all_dataCa = pd.concat([trainCa, testCa])

all_dataCa['ATA'] = pd.to_datetime(all_dataCa['ATA'])
all_dataCa['Year'] = all_dataCa['ATA'].dt.year
all_dataCa['Month'] = all_dataCa['ATA'].dt.month
all_dataCa['Day'] = all_dataCa['ATA'].dt.day
all_dataCa['Hour'] = all_dataCa['ATA'].dt.hour
all_dataCa['DayofWeek'] = all_dataCa['ATA'].dt.dayofweek
all_dataCa['Date'] = all_dataCa['ATA'].dt.date
all_dataCa['SHIP'] = all_dataCa['BREADTH'] * all_dataCa['DEPTH'] * all_dataCa['LENGTH']

all_dataCa_2 = all_dataCa.drop(columns = ['SAMPLE_ID', 'ATA', 'CI_HOUR'])

le = LabelEncoder()

cat = all_dataCa_2.columns[all_dataCa_2.dtypes == 'object']

cat

for i in cat:
    
    all_dataCa_2[i] = le.fit_transform(all_dataCa_2[i])

trainCa_2 = all_data_2[:len(trainCa)]

testCa_2 = all_data_2[len(trainCa):]

qcut = pd.qcut(trainCa['CI_HOUR'], q = 3, duplicates='drop')
qcut = le.fit_transform(qcut)

skf = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)

result = 0

train_index_list = []
val_index_list = []

for train_index, val_index in skf.split(train_2, qcut):  
    
    X_train = trainCa_2.iloc[train_index]
    y_train = trainCa['CI_HOUR'].iloc[train_index]**(1/4)
    
    X_val = trainCa_2.iloc[val_index]
    y_val = trainCa['CI_HOUR'].iloc[val_index]**(1/4)
    
    cbc = CatBoostRegressor(verbose = 100, iterations = 20000, learning_rate = 0.05, eval_metric = 'MAE')
    
    cbc.fit(X_train, y_train, eval_set = (X_val, y_val), early_stopping_rounds = 100)
      
    train_index_list.append(train_index)
    val_index_list.append(val_index)
    
    result += cbc.predict(testCa_2)**(4) / 10

cbr = CatBoostRegressor(verbose = 100, eval_metric = 'MAE', iterations = 1000, learning_rate = 0.05) # iterations = 10000, learning_rate = 0.05 
cbr.fit(trainCa_2, testCa_2, early_stopping_rounds = 100)

## ============================================================

In [123]:
all_data = pd.concat([train, test])

In [124]:
all_data['ATA'] = pd.to_datetime(all_data['ATA'])
all_data['Year'] = all_data['ATA'].dt.year
all_data['Month'] = all_data['ATA'].dt.month
all_data['Day'] = all_data['ATA'].dt.day
all_data['Hour'] = all_data['ATA'].dt.hour
all_data['DayofWeek'] = all_data['ATA'].dt.dayofweek
all_data['Date'] = all_data['ATA'].dt.date
all_data['SHIP'] = all_data['BREADTH'] * all_data['DEPTH'] * all_data['LENGTH']

In [125]:
all_data['NEW_COLUMN'] = all_data.apply(lambda row: str((row['SHIP_TYPE_CATEGORY'], row['ARI_CO'])), axis=1)

In [126]:
all_data_2 = all_data.drop(columns = ['SAMPLE_ID', 'ATA', 'CI_HOUR'])

In [127]:
numeric_columns = all_data_2.select_dtypes(include=['float64', 'int64', 'int32']).columns

In [128]:
# StandardScaler 객체 생성
scaler = StandardScaler()

# 수치형 칼럼들에 대해 표준화 수행
all_data_2[numeric_columns] = scaler.fit_transform(all_data_2[numeric_columns])
all_data_2

,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ID,BREADTH,BUILT,DEADWEIGHT,DEPTH,DRAUGHT,...,ATA_LT,PORT_SIZE,Year,Month,Day,Hour,DayofWeek,Date,SHIP,NEW_COLUMN
0,SG,GIW5,Container,0.373937,Z618338,-0.049544,1.312253,-0.618099,-1.070755,-0.236341,...,-1.029300,2.074745,-0.449447,1.608015,0.144405,1.431210,-1.320979,2018-12-17,-0.631774,"('Container', 'SG')"
1,IN,UJM2,Bulk,-0.624796,X886125,-0.049544,-0.248044,-0.435387,-1.070755,-0.236341,...,0.024102,-0.787392,-2.105227,0.740101,0.829552,-0.707420,-0.771517,2014-09-23,-0.631774,"('Bulk', 'IN')"
2,CN,EUC8,Container,-0.624796,T674582,1.776749,-0.389889,1.298805,2.238244,1.939646,...,-0.878814,0.880102,-1.691282,-1.285033,-1.454270,1.573785,-0.771517,2015-02-03,3.098935,"('Container', 'CN')"
3,JP,ZAG4,Container,-0.624796,Y847238,-0.962690,0.461182,-0.892010,-1.070755,-0.236341,...,0.174588,-0.620946,0.378443,-1.574338,0.144405,-0.992571,0.876869,2020-01-17,-0.855170,"('Container', 'JP')"
4,SG,GIW5,Container,0.249637,A872328,1.776749,-0.673579,0.826273,0.583744,-0.236341,...,0.475560,2.074745,0.378443,-1.574338,1.172125,-0.564845,1.975793,2020-01-26,1.200071,"('Container', 'SG')"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220486,BR,TMW2,Bulk,0.077422,E731427,2.689895,0.035647,3.692967,2.238244,1.939646,...,0.174588,-0.950397,-0.863392,-0.706423,-1.340079,0.575758,-0.771517,2017-04-04,3.389350,"('Bulk', 'BR')"
220487,CA,GRQ5,Bulk,-0.467278,Q434522,-0.049544,-0.815424,0.267110,0.583744,-0.236341,...,0.174588,-0.999674,-1.277337,0.450796,0.715361,0.718333,-1.320979,2016-08-22,-0.050945,"('Bulk', 'CA')"
220488,JP,VYJ1,Tanker,-0.072785,U148516,-1.875836,-0.106198,-0.981949,-2.725254,-2.412328,...,1.679448,-0.730514,1.206333,0.161491,-0.654932,0.433182,1.975793,2022-07-10,-1.033886,"('Tanker', 'JP')"
220489,CN,QQW1,Cargo,-0.624796,C235264,-1.875836,1.737789,-0.946982,-1.070755,-2.412328,...,1.528962,-0.336291,0.378443,1.608015,1.400507,0.433182,-1.320979,2020-12-28,-0.974314,"('Cargo', 'CN')"


In [129]:
le = LabelEncoder()
all_data_2['ID'] = le.fit_transform(all_data_2['ID'])
all_data_2['SHIPMANAGER'] = le.fit_transform(all_data_2['SHIPMANAGER'])
all_data_2['Date'] = le.fit_transform(all_data_2['Date'])

all_data_2['U_WIND'] = all_data_2['U_WIND'].fillna(all_data_2['U_WIND'].mean())
all_data_2['V_WIND'] = all_data_2['V_WIND'].fillna(all_data_2['V_WIND'].mean())
all_data_2['BN'] = all_data_2['BN'].fillna(all_data_2['BN'].mean())
all_data_2['AIR_TEMPERATURE'] = all_data_2['AIR_TEMPERATURE'].fillna(all_data_2['AIR_TEMPERATURE'].mean())

In [130]:
all_data_2.isna().sum()

ARI_CO                0
ARI_PO                0
SHIP_TYPE_CATEGORY    0
DIST                  0
ID                    0
BREADTH               0
BUILT                 0
DEADWEIGHT            0
DEPTH                 0
DRAUGHT               0
GT                    0
LENGTH                0
SHIPMANAGER           0
FLAG                  0
U_WIND                0
V_WIND                0
AIR_TEMPERATURE       0
BN                    0
ATA_LT                0
PORT_SIZE             0
Year                  0
Month                 0
Day                   0
Hour                  0
DayofWeek             0
Date                  0
SHIP                  0
NEW_COLUMN            0
dtype: int64

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

cat = all_data_2.columns[all_data_2.dtypes == 'object']

cat

In [131]:
cat = ['ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'SHIPMANAGER', 'FLAG',
       'Date', 'NEW_COLUMN']

In [132]:
all_data_2 = pd.get_dummies(all_data_2)
all_data_2

,DIST,ID,BREADTH,BUILT,DEADWEIGHT,DEPTH,DRAUGHT,GT,LENGTH,SHIPMANAGER,...,"NEW_COLUMN_('Tanker', 'MZ')","NEW_COLUMN_('Tanker', 'PE')","NEW_COLUMN_('Tanker', 'QA')","NEW_COLUMN_('Tanker', 'RU')","NEW_COLUMN_('Tanker', 'SG')","NEW_COLUMN_('Tanker', 'TT')","NEW_COLUMN_('Tanker', 'TW')","NEW_COLUMN_('Tanker', 'UA')","NEW_COLUMN_('Tanker', 'US')","NEW_COLUMN_('Tanker', 'VN')"
0,0.373937,26361,-0.049544,1.312253,-0.618099,-1.070755,-0.236341,-0.626931,-0.330343,711,...,False,False,False,False,False,False,False,False,False,False
1,-0.624796,24690,-0.049544,-0.248044,-0.435387,-1.070755,-0.236341,-0.453735,-0.330343,4971,...,False,False,False,False,False,False,False,False,False,False
2,-0.624796,20310,1.776749,-0.389889,1.298805,2.238244,1.939646,2.513527,2.174543,1436,...,False,False,False,False,False,False,False,False,False,False
3,-0.624796,25655,-0.962690,0.461182,-0.892010,-1.070755,-0.236341,-0.914743,-1.121359,4006,...,False,False,False,False,False,False,False,False,False,False
4,0.249637,972,1.776749,-0.673579,0.826273,0.583744,-0.236341,1.408127,1.251690,1770,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220486,0.077422,4879,2.689895,0.035647,3.692967,2.238244,1.939646,2.793698,1.647199,5586,...,False,False,False,False,False,False,False,False,False,False
220487,-0.467278,16919,-0.049544,-0.815424,0.267110,0.583744,-0.236341,0.035290,0.197002,281,...,False,False,False,False,False,False,False,False,False,False
220488,-0.072785,20672,-1.875836,-0.106198,-0.981949,-2.725254,-2.412328,-1.039546,-1.912376,1057,...,False,False,False,False,False,False,False,False,False,False
220489,-0.624796,2222,-1.875836,1.737789,-0.946982,-1.070755,-2.412328,-1.001341,-1.648704,5750,...,False,False,False,False,False,False,False,False,False,False


In [133]:
train_2 = all_data_2[:len(train)]

test_2 = all_data_2[len(train):]

In [134]:
knn_regressor = KNeighborsRegressor(n_neighbors=5)

knn_regressor.fit(train_2, train['CI_HOUR']**(1/5))

KNeighborsRegressor()

In [135]:
print(mae)

11.182276290787682


In [136]:
result = knn_regressor.predict(test_2)**5

In [137]:
result[result<0] = 0
result[result>1500] = 1500

In [138]:
sub['CI_HOUR'] = result
sub

,SAMPLE_ID,CI_HOUR
0,TEST_000000,0.020688
1,TEST_000001,1.907798
2,TEST_000002,57.638639
3,TEST_000003,0.471112
4,TEST_000004,0.034724
...,...,...
220486,TEST_220486,67.255210
220487,TEST_220487,7.360465
220488,TEST_220488,0.033375
220489,TEST_220489,0.142727


In [141]:
sub.to_csv('knn2nd.csv', index=0)

In [140]:
setup(data = train_2, target = train['CI_HOUR']**(1/5))

NameError: name 'setup' is not defined

In [ ]:
best_model = compare_models(fold = 4)

In [ ]:
qcut = pd.qcut(train['CI_HOUR'], q = 3, duplicates='drop')
qcut = le.fit_transform(qcut)

In [ ]:
from catboost import CatBoostRegressor
from sklearn.model_selection import StratifiedKFold
import warnings

warnings.filterwarnings('ignore')

skf = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)

result = 0

train_index_list = []
val_index_list = []

for train_index, val_index in skf.split(train_2, qcut):  
    
    X_train = train_2.iloc[train_index]
    y_train = train['CI_HOUR'].iloc[train_index]**(1/4)
    
    X_val = train_2.iloc[val_index]
    y_val = train['CI_HOUR'].iloc[val_index]**(1/4)
    
    cbc = CatBoostRegressor(verbose = 100, iterations = 20000, learning_rate = 0.05, eval_metric = 'MAE')
    
    cbc.fit(X_train, y_train, eval_set = (X_val, y_val), early_stopping_rounds = 100)
      
    train_index_list.append(train_index)
    val_index_list.append(val_index)
    
    result += cbc.predict(test_2)**(4) / 10

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(train_2, train['CI_HOUR']**(1/4), test_size = 0.2, random_state = 4432)

from catboost import CatBoostRegressor

cbr = CatBoostRegressor(verbose = 100, eval_metric = 'MAE', iterations = 10000, learning_rate = 0.05) # iterations = 10000, learning_rate = 0.05 
cbr.fit(x_train, y_train, eval_set = (x_valid, y_valid), early_stopping_rounds = 100)

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
valpred = cbr.predict(x_valid)**4
valpred[valpred<0] = 0
mean_absolute_error(y_valid, valpred)

In [ ]:
valpred[valpred > 1500]

In [ ]:
valpred = cbc.predict(x_valid)**4
valpred[valpred > 1500] = 1500
mean_absolute_error(y_valid, valpred)

In [ ]:
train['CI_HOUR'].sort_values(ascending = False)[:100]

In [ ]:
train.groupby('SHIP_TYPE_CATEGORY')['CI_HOUR'].mean()

result = cbr.predict(test_2)**(4)

In [ ]:
resultforsaving = result

In [ ]:
resultforsaving[resultforsaving < 0] = 0
resultforsaving[resultforsaving > 1500] = 1500

In [ ]:
sub['CI_HOUR'] = resultforsaving

In [ ]:
sub.to_csv('sub0137.csv', index=0)